
 # Comprehensive Exam

 ## Coding Artifact

 Kalin Gibbons

 Nov 20, 2020
 ### Data Description

 This _training_ dataset is generated from simplified finite element models of a
 cruciate-sacrificing, post and cam driven knee implant performing a deep-knee-bend.
 The implant geometries and surgical alignments are parameterized by 13 predictor
 variables which were drawn using Latin hypercube sampling from a range of currently
 used manufacturer dimensions, and angles performed during successful surgeries. There
 were originally 15 predictors for this dataset, but two were fixed at average values
 for this particular batch of simulations. For the test dataset, the same predictors
 were uniformly drawn across the ranges of potential values.
 ### Data cleaning

In [1]:
import logging

In [2]:
import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as spio
import seaborn as sns

from fepredict.datasets import select_by_regex

%load_ext autoreload
%autoreload 2


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [3]:
mpl.rcParams["mathtext.fontset"] = "stix"
mpl.rcParams["font.size"] = 14
mpl.rcParams["font.family"] = "Times New Roman"
plt.rcParams["figure.figsize"] = (9, 5.5)

sns.set_context("poster")
sns.set(rc={"figure.figsize": (16, 9.0)})
sns.set_style("whitegrid")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)


 ## Data Cleaning

 ---

 The data is contained within `MATLAB` binary files, which are easily imported into a
 `pandas` `DataFrame`. Some of the sensors implemented within the FEA model that
 generated the data do not consistently activate during a deep-knee-bend, so those
 columns need to be filtered out. Finally, if the uniform feature draw resulted in a
 particularly infeasible implant geometry, the simulation would fail, producing no
 results. These failed simulations will need to be removed.

 ### Controlling variable and function definitions

In [4]:
drop_regex = [
    # r'^time$',
    r"(femfe|patthick|patml|patsi)",  # features held constant
    r"^\w{3}_[xyz]_\w{3,4}$",
    r"^post\w+",
    r"^v[ilm][2-6]_(disp|force)$",
    r"^v[lm]1_(disp|force)$",
    r"^vert_(disp|force)$",
    r"^flex_(force|rot)$",
    r"^ap_force$",
    r"^(vv|ie)_torque",
    r"^(ml|pcm|pcl|pol)_force$",  # Always zero
    r"^(lclp|lcl|pmc|lcla|mcla)_force$",  # Often zero and bad predict
    r"^(pom|alc|mcl|mclp)_force$",  # Often zero and fairly bad predict.
]


def import_matlab_data(matfilepath):
    """Import MATLAB .mat binary into a panda DataFrame.

    Args:
        matfilepath (str|pathlike): The path to the .mat file

    Returns:
        pandas.DataFrame: A tabulation of features and response data, where
            each response entry is an array of data points for each step in
            the simulation.
    """
    data = spio.loadmat(matfilepath, squeeze_me=True)
    keys = list(data.keys())
    data = data[keys[-1]]
    columns = list(map(lambda x: x.lower(), data.dtype.names))
    old = ["femie", "femvv", "tibslope", "tibie", "tibvv", "xn", "ctf", "ctm"]
    new = [
        "fem_ie",
        "fem_vv",
        "tib_slope",
        "tib_ie",
        "tib_vv",
        "cop_",
        "force_",
        "torque_",
    ]
    for o, n in zip(old, new):
        columns = list(map(lambda x: x.replace(o, n), columns))

    data_df = pd.DataFrame(data)
    data_df.columns = columns
    return data_df


def drop_columns(data_df, regex_list):
    # """Remove columns using regular expressions."""
    return select_by_regex(data_df, regex_list, axis=1, negate=True)


def remove_failed(response_series, df_list):
    """Remove rows of DataFrames selected by empty entries in a series.

    Able to handle multiple DataFrames, allowing for removing empty rows after
    splitting into feature/response DataFrames.

    Args:
        response_series (pandas.Series): A series representing a sample column
            from the DataFrame to be split.
        df_list ([pandas.DataFrames]): A list of dataframes with shared indices
            that need empty rows removed.

    Returns:
        [pandas.DataFrames]: The list of dataframes (or single dataframe) with
            empty rows removed.
    """
    try:
        len(df_list)
    except TypeError:
        df_list = [df_list]

    failed_idx = response_series.apply(lambda x: x.size == 0)
    new_df_list = np.full(len(df_list), np.nan, dtype="object")
    for idx, df in enumerate(df_list):
        new_df_list[idx] = df.loc[~failed_idx]

    if len(new_df_list) == 1:
        return new_df_list[0]
    else:
        return new_df_list

 ### Locating the data

 The `MATLAB` MAT files are stored in the `data/interim` folder because the raw data
 was stored in plaintext CSV files after being extracted from the FEA simulations.
 Once cleaned, we'll store the cleaned data in `data/preprocessed`.

In [5]:
# Source paths
# dirty_data_dir = Path.cwd().parent / "data" / "interim"
dirty_data_dir = Path.cwd().parent / "data" / "interim"
# print((dirty_data_dir)
dirty_test_path = dirty_data_dir / "test.mat"
dirty_train_path = dirty_data_dir / "doe.mat"

# Destination paths
cleaned_dir = dirty_data_dir.parent / "preprocessed"
cleaned_test_path = cleaned_dir / "test.parquet"
cleaned_train_path = cleaned_dir / "train.parquet"
print(dirty_test_path)

c:\Users\kalingibbons\Documents\git-projects\sd-stress-predictor\data\interim\test.mat


 ### Data import and cleaning

 Reading the MAT file tables into memory, and outputting zpart of the dataframes to
 take a look at the data, then dropping the extraneous columns and taking a look at
 the results. We'll only look at the results from the testing set, but run identical
 operations on the training set, as well.

In [6]:
# Test
dirty_test = import_matlab_data(dirty_test_path)
dirty_test.shape

(100, 170)

In [7]:
dirty_test.head()

,fem_rad_dist,fem_rad_post,fem_rad_cor,tib_ant_conf,tib_post_conf,tib_cor_conf,troch_ang,troch_offset,fem_ie,fem_vv,tib_slope,tib_ie,tib_vv,cam_rad,femfe,patthick,patml,patsi,time,medial_area,lateral_area,post_area,patella_area,tib_x_orig,tib_y_orig,tib_z_orig,tib_x_med,tib_y_med,tib_z_med,tib_x_sup,tib_y_sup,tib_z_sup,tib_x_ant,tib_y_ant,tib_z_ant,fem_x_orig,fem_y_orig,fem_z_orig,fem_x_med,fem_y_med,fem_z_med,fem_x_sup,fem_y_sup,fem_z_sup,fem_x_ant,fem_y_ant,fem_z_ant,pat_x_orig,pat_y_orig,pat_z_orig,pat_x_med,pat_y_med,pat_z_med,pat_x_sup,pat_y_sup,pat_z_sup,pat_x_ant,pat_y_ant,pat_z_ant,post_cop_1,...,lcla_disp,lcla_force,lclp_disp,lclp_force,pfl_disp,pfl_force,alc_disp,alc_force,pcm_disp,pcm_force,pcl_disp,pcl_force,pol_disp,pol_force,pom_disp,pom_force,pmc_disp,pmc_force,vi1_disp,vi1_force,vi2_disp,vi2_force,vi3_disp,vi3_force,vi4_disp,vi4_force,vi5_disp,vi5_force,vi6_disp,vi6_force,vl1_disp,vl1_force,vl2_disp,vl2_force,vl3_disp,vl3_force,vl4_disp,vl4_force,vm1_disp,vm1_force,vm2_disp,vm2_force,vm3_disp,vm3_force,vm4_disp,vm4_force,pl_disp,pl_force,pat_fem_flexion,pat_fem_valgus,pat_fem_external,pat_fem_lat,pat_fem_ant,pat_fem_inf,tib_fem_flexion,tib_fem_valgus,tib_fem_external,tib_fem_lat,tib_fem_ant,tib_fem_inf
0,44.44,24.87,27.89,0.2,0.51,0.64,15.51,0.35,3.63,7.1,7.07,10.02,2.73,35,0,0,0,0,"[0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0....","[87.671, 82.012, 85.435, 81.799, 90.488, 84.27...","[79.9, 95.764, 70.936, 82.842, 72.962, 72.633,...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[36.935, 52.068, 37.329, 43.279, 39.378, 47.61...","[-0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, ...","[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.97, 0.971, 0.969, 0.967, 0.969, 0.975, 0.98...","[-0.091, -0.092, -0.09, -0.092, -0.092, -0.09,...","[-0.225, -0.22, -0.228, -0.238, -0.228, -0.202...","[0.088, 0.09, 0.088, 0.089, 0.089, 0.088, 0.08...","[0.996, 0.996, 0.996, 0.996, 0.996, 0.996, 0.9...","[-0.02, -0.02, -0.021, -0.022, -0.021, -0.018,...","[0.226, 0.221, 0.229, 0.239, 0.229, 0.203, 0.1...","[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0....","[0.974, 0.975, 0.973, 0.971, 0.974, 0.979, 0.9...","[1.18, 1.194, 1.335, 1.176, 1.2, 1.391, 1.189,...","[26.976, 26.924, 26.879, 26.869, 26.907, 26.84...","[-6.818, -6.367, -7.143, -6.779, -5.943, -6.26...","[2.18, 2.194, 2.335, 2.176, 2.2, 2.391, 2.189,...","[26.976, 26.924, 26.879, 26.869, 26.907, 26.84...","[-6.818, -6.367, -7.143, -6.779, -5.943, -6.26...","[1.18, 1.194, 1.335, 1.176, 1.2, 1.391, 1.189,...","[27.943, 27.887, 27.841, 27.828, 27.866, 27.79...","[-7.073, -6.636, -7.414, -7.06, -6.228, -6.552...","[1.18, 1.194, 1.335, 1.176, 1.2, 1.391, 1.189,...","[27.231, 27.194, 27.151, 27.149, 27.192, 27.13...","[-5.851, -5.403, -6.18, -5.82, -4.984, -5.305,...","[-2.801, -3.039, -2.613, -2.862, -2.595, -2.67...","[44.901, 44.985, 44.781, 44.955, 44.799, 44.64...","[33.253, 33.561, 32.767, 33.001, 33.806, 33.5,...","[-1.822, -2.063, -1.635, -1.884, -1.616, -1.69...","[44.815, 44.902, 44.692, 44.868, 44.712, 44.55...","[33.066, 33.359, 32.579, 32.809, 33.627, 33.30...","[-2.727, -2.97, -2.539, -2.79, -2.521, -2.599,...","[45.896, 45.98, 45.774, 45.949, 45.793, 45.637...","[33.18, 33.486, 32.682, 32.918, 33.729, 33.416...","[-2.608, -2.832, -2.418, -2.664, -2.41, -2.472...","[44.959, 45.043, 44.85, 45.023, 44.862, 44.711...","[34.233, 34.537, 33.746, 33.979, 34.787, 34.47...","[2.054, 2.066, 2.053, 2.065, 2.064, 2.053, 2.0...",...,"[-9.751, -9.941, -9.881, -10.016, -10.018, -10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-10.162, -10.43, -10.338, -10.499, -10.562, -...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-9.638, -9.535, -9.665, -9.731, -9.433, -9.20...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-3.107, -3.577, -3.046, -3.151, -3.691, -4.03...","[0.88, 0.0, 1.078, 0.739, 0.0, 0.0, 0.0, 0.0, ...","[-4.936, -5.3, -5.283, -5.455, -5.613, -5.88, ...","[0, 0, 0, 0, 0, 0, 0, 0,

In [8]:
dirty_test, dropped_cols = drop_columns(dirty_test, drop_regex)
print("Dropped", dropped_cols)

Dropped Index(['femfe', 'patthick', 'patml', 'patsi', 'post_area', 'tib_x_orig',
       'tib_y_orig', 'tib_z_orig', 'tib_x_med', 'tib_y_med', 'tib_z_med',
       'tib_x_sup', 'tib_y_sup', 'tib_z_sup', 'tib_x_ant', 'tib_y_ant',
       'tib_z_ant', 'fem_x_orig', 'fem_y_orig', 'fem_z_orig', 'fem_x_med',
       'fem_y_med', 'fem_z_med', 'fem_x_sup', 'fem_y_sup', 'fem_z_sup',
       'fem_x_ant', 'fem_y_ant', 'fem_z_ant', 'pat_x_orig', 'pat_y_orig',
       'pat_z_orig', 'pat_x_med', 'pat_y_med', 'pat_z_med', 'pat_x_sup',
       'pat_y_sup', 'pat_z_sup', 'pat_x_ant', 'pat_y_ant', 'pat_z_ant',
       'post_cop_1', 'post_cop_2', 'post_cop_3', 'post_press', 'ie_torque',
       'vert_disp', 'vert_force', 'ap_force', 'vv_torque', 'ml_force',
       'flex_rot', 'flex_force', 'post_force_1', 'post_force_2',
       'post_torque_1', 'post_torque_2', 'mcl_force', 'mcla_force',
       'mclp_force', 'lcl_force', 'lcla_force', 'lclp_force', 'alc_force',
       'pcm_force', 'pcl_force', 'pol_force', 'pom_f

In [9]:
clean_test = remove_failed(dirty_test.iloc[:, -1], [dirty_test])
clean_test.shape

(87, 75)

In [10]:
clean_test.head()

,fem_rad_dist,fem_rad_post,fem_rad_cor,tib_ant_conf,tib_post_conf,tib_cor_conf,troch_ang,troch_offset,fem_ie,fem_vv,tib_slope,tib_ie,tib_vv,cam_rad,time,medial_area,lateral_area,patella_area,med_cop_1,med_cop_2,med_cop_3,lat_cop_1,lat_cop_2,lat_cop_3,pat_cop_1,pat_cop_2,pat_cop_3,med_press,lat_press,pat_press,ie_rot,ap_disp,vv_rot,ml_disp,med_force_1,med_force_2,med_torque_1,med_torque_2,lat_force_1,lat_force_2,lat_torque_1,lat_torque_2,pat_force_1,pat_force_2,pat_force_3,mcl_disp,mcla_disp,mclp_disp,lcl_disp,lcla_disp,lclp_disp,pfl_disp,pfl_force,alc_disp,pcm_disp,pcl_disp,pol_disp,pom_disp,pmc_disp,vi1_disp,vi1_force,pl_disp,pl_force,pat_fem_flexion,pat_fem_valgus,pat_fem_external,pat_fem_lat,pat_fem_ant,pat_fem_inf,tib_fem_flexion,tib_fem_valgus,tib_fem_external,tib_fem_lat,tib_fem_ant,tib_fem_inf
0,44.44,24.87,27.89,0.2,0.51,0.64,15.51,0.35,3.63,7.1,7.07,10.02,2.73,35,"[0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0....","[87.671, 82.012, 85.435, 81.799, 90.488, 84.27...","[79.9, 95.764, 70.936, 82.842, 72.962, 72.633,...","[36.935, 52.068, 37.329, 43.279, 39.378, 47.61...","[24.104, 23.967, 24.506, 24.015, 23.994, 24.49...","[-2.601, -2.6, -2.647, -2.611, -2.542, -2.622,...","[-1.135, -0.543, -1.834, -0.871, 0.358, -1.132...","[-20.86, -20.988, -20.4, -20.927, -21.004, -20...","[3.599, 3.632, 3.611, 3.664, 3.625, 3.534, 3.6...","[-0.69, -0.042, -1.748, -1.259, 0.39, -0.119, ...","[-4.489, -4.65, -4.02, -4.253, -3.945, -4.149,...","[42.782, 42.845, 42.758, 42.897, 42.779, 42.64...","[31.633, 31.951, 31.029, 31.287, 32.06, 31.782...","[19.202, 17.849, 19.071, 19.966, 19.932, 19.83...","[18.819, 22.559, 16.401, 18.859, 17.772, 16.40...","[37.981, 45.328, 36.356, 40.552, 37.713, 41.23...","[0.228, 0.222, 0.231, 0.241, 0.231, 0.205, 0.1...","[-5.543, -5.02, -5.785, -5.376, -4.518, -4.814...","[-0.091, -0.092, -0.091, -0.092, -0.092, -0.09...","[1.18, 1.194, 1.335, 1.176, 1.2, 1.391, 1.189,...","[696.462, 623.766, 709.752, 713.19, 767.876, 7...","[47.408, 36.156, 52.852, 40.167, 33.795, 68.99...","[78.721, 334.106, -10.923, -88.766, -98.118, 8...","[-1022.336, 760.126, -1623.193, -1204.934, -22...","[648.2, 949.36, 480.232, 655.984, 525.885, 448...","[69.204, 90.227, 57.666, 66.546, 35.561, 41.89...","[-192.365, 39.887, -254.946, -325.046, -252.78...","[2063.345, 4665.438, 870.765, 2099.507, 672.64...","[475.898, 576.646, 437.015, 523.144, 500.194, ...","[-31.786, -43.63, -15.841, -24.61, -26.518, -2...","[58.292, 75.748, 47.876, 67.073, 63.904, 56.77...","[0.195, 0.126, 0.147, 0.046, -0.007, 0.043, -0...","[1.531, 1.573, 1.576, 1.549, 1.561, 1.628, 1.6...","[-0.765, -0.915, -0.881, -1.033, -1.134, -1.10...","[-9.318, -9.572, -9.451, -9.597, -9.679, -9.75...","[-9.751, -9.941, -9.881, -10.016, -10.018, -10...","[-10.162, -10.43, -10.338, -10.499, -10.562, -...","[-9.638, -9.535, -9.665, -9.731, -9.433, -9.20...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-3.107, -3.577, -3.046, -3.151, -3.691, -4.03...","[-4.936, -5.3, -5.283, -5.455, -5.613, -5.88, ...","[-7.468, -7.996, -7.72, -7.972, -8.363, -8.601...","[-9.612, -9.901, -10.1, -10.357, -10.371, -10....","[-2.227, -2.609, -2.289, -2.469, -2.784, -3.05...","[0.625, 0.505, 0.564, 0.6, 0.557, 0.182, -0.20...","[4.541, 4.801, 5.356, 5.427, 6.0, 6.219, 6.52,...","[0.0, 79.285, 55.016, 68.598, 62.226, 59.519, ...","[1.102, 1.226, 1.031, 1.141, 1.088, 1.067, 1.1...","[138.235, 176.889, 115.874, 150.25, 133.782, 1...","[10.577, 11.296, 10.84, 11.558, 12.122, 12.028...","[4.242, 3.956, 4.246, 4.129, 4.245, 4.185, 4.0...","[-11.111, -11.984, -11.241, -11.448, -10.682, ...","[3.981, 4.233, 3.948, 4.038, 3.795, 4.063, 4.0...","[41.275, 41.173, 41.291, 41.147, 41.012, 41.12...","[-14.61, -14.681, -14.101, -14.384, -14.447, -...","[13.622, 14.496, 14.565, 15.011, 15.343, 15.82...","[5.048, 5.162, 5.048, 5.105, 5.105, 5.048, 5.0...","[-13.109, -12.817, -13.295, -13.884, -13.295, ...","[1.18, 1.194, 1.335, 1.176, 1.2, 1.391, 1.189,...","[6.275, 5.825, 6.575, 6.184, 5.374, 5.776, 5.0

In [11]:
# Train
dirty_train = import_matlab_data(dirty_train_path)
dirty_train, _ = drop_columns(dirty_train, drop_regex)
clean_train = remove_failed(dirty_train.iloc[:, -1], [dirty_train])

 ## Save the cleaned data
 Everything looked great, so we can save the cleaned data.
 %
 You'll likely have to modify the dataframes to use MultiIndexing, since to_parquet now
 dislikes the "object" datatype. MultiIndexing is pandas' way of dealing with more than
 2D data. I added the old preprocessed data to the folders, but didn't check if loading
 them would work.
 clean_test.to_parquet(cleaned_test_path)
 clean_train.to_parquet(cleaned_train_path)